In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
class SE(nn.Module):
    def __init__(self, in_ch):
        super(SE, self).__init__()
        self.se = nn.Sequential(
            nn.Linear(in_ch, in_ch // 8, bias=False),
            nn.ReLU(),
            nn.Linear(in_ch // 8, in_ch, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        if len(x.size()) > 3:
            x = F.adaptive_avg_pool2d(x, 1).squeeze()
        sc = x
        #print(sc.shape)
        return self.se(x) * sc + sc

In [11]:
## 两层卷积的cbr
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''

    def __init__(self, in_ch, out_ch, stride=1, use_se=False, bottleneck=False):
        super(double_conv, self).__init__()
        self.use_se = use_se
        self.bottleneck = bottleneck

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch)
        )

        if self.use_se:
            self.se = SE(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        sc = x
        x = self.conv2(x)

        if self.use_se:
            x = self.se(x)

        x += sc
        x = F.relu(x, inplace=True)
        return x

In [5]:
# 通道变换操作
class inconv(nn.Module):
    '''
    inconv only changes the number of channels
    '''
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x

In [22]:
## 下采样 
class down(nn.Module):
    def __init__(self, in_ch, out_ch, use_se=False):
        super(down, self).__init__()
        self.mpconv = double_conv(in_ch, out_ch, stride=2, use_se=use_se)

    def forward(self, x):
        x = self.mpconv(x)
        return x

In [26]:
x = torch.randn(1,32,64,64)
x.shape
model_d = down(32, 32)
out = model_d(x)
out.shape

torch.Size([1, 32, 32, 32])

In [ ]:
out2 = model_d(x)
out2.shape#256->128

torch.Size([1, 64, 128, 128])

In [36]:

# upsample
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''

    def __init__(self, in_ch, out_ch, stride=1, use_se=False, bottleneck=False):
        super(double_conv, self).__init__()
        self.use_se = use_se
        self.bottleneck = bottleneck

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch)
        )

        if self.use_se:
            self.se = SE(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        sc = x
        x = self.conv2(x)

        if self.use_se:
            x = self.se(x)

        x += sc
        x = F.relu(x, inplace=True)
        return x

class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False, op="none", use_se=False):
        super(up, self).__init__()
        self.bilinear = bilinear
        self.op = op
        self.mixup_ratio = 0.95
        assert op in ["concat", "none", "add", 'mix']

        self.conv = double_conv(in_ch, out_ch, use_se=use_se)

    def forward(self, x1, x2=None):
        if x2 is not None:#非空
            if torch.is_tensor(x2):
                x1 = F.interpolate(x1, x2.size()[-2:], mode='bilinear', align_corners=False)
            else:
                x1 = F.interpolate(x1, x2, mode='bilinear', align_corners=False)
        else:
            x1 = F.interpolate(x1, scale_factor=2,  mode='bilinear', align_corners=False)

        if self.op == "concat":
            x = torch.cat([x2, x1], dim=1)
        elif self.op == 'add':
            x = x1 + x2
        else:
            x = x1
            
        x = self.conv(x)

        return x

In [42]:
x1 = torch.randn(1, 32, 128, 128)
x2 = torch.randn(1, 64, 128, 128) 
x1.shape


torch.Size([1, 32, 128, 128])

In [49]:
up_model = up(32,64)
out1 = up_model(x1)
out1.shape

torch.Size([1, 64, 256, 256])

In [43]:
up_model = up(96,64,op="concat")
# up_model.shape

In [46]:
out = up_model(x1,x2)
out.shape

torch.Size([1, 64, 128, 128])

In [5]:
a = 10
re = {'a': 0., 'b':0.}
re['a'] += a
re

{'a': 10.0, 'b': 0.0}

In [3]:
epoch = 200
a = (epoch - 200) / (400 - 200)
a

0.0

In [8]:
epoch = 500
b = (epoch - 400) / (600 - 400)
b

0.5

In [27]:
class inconv(nn.Module):
    '''
    inconv only changes the number of channels
    '''

    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x

In [32]:
x = torch.randn(1,1,128,128)
x.shape

torch.Size([1, 1, 128, 128])

In [33]:
model_inconv = inconv(1, 32)
out = model_inconv(x)
out.shape

torch.Size([1, 32, 128, 128])

In [34]:
print(model_inconv)

inconv(
  (conv): double_conv(
    (conv1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (conv2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)


In [35]:
for i in range(4):
    print(i)
    

0
1
2
3


In [2]:
memory_channel = 128 * 4 * 4 
attention = nn.MultiheadAttention(memory_channel, 8)
# attn_output, attn_output_weights = attention(query, key, value)


In [6]:
import torch
import torch.nn as nn

# 假设我们有一个序列长度为10，嵌入维度为512，我们想要实现8个头的多头注意力
sequence_length = 10
embedding_dim = 512
num_heads = 8
batch_size = 32

# 实例化多头注意力模块
multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

# 创建一些随机数据来模拟输入
# query, key, value 通常是一样的，但也可以是不同的
query = torch.rand(batch_size, sequence_length, embedding_dim)
key = torch.rand(batch_size, sequence_length, embedding_dim)
value = torch.rand(batch_size, sequence_length, embedding_dim)

# 应用多头注意力
attn_output, attn_output_weights = multihead_attn(query, key, value)

# 输出的形状将是 (batch_size, sequence_length, embed_dim)
print(query.shape)
print(attn_output.shape)

# 输出权重的形状将是 (batch_size, num_heads, sequence_length, sequence_length)
print(attn_output_weights.shape)

torch.Size([32, 10, 512])
torch.Size([32, 10, 512])
torch.Size([10, 32, 32])


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 假设我们的图像是224x224，我们将其分割成16个14x14的块
image_size = 224
patch_size =14
num_patches = (image_size // patch_size) ** 2
embedding_dim = 512
num_heads = 8
batch_size = 1


# 将图像分割成块，并线性嵌入
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=14, in_chans=3, 
    embed_dim=512):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(
            in_chans, embed_dim, kernel_size=patch_size, 
            stride=patch_size
        )

    def forward(self, x):
        x = self.proj(x)  # (B, E, H, W)
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x


# 实例化图像分割和嵌入模块
patch_embed = PatchEmbed(
    img_size=image_size, patch_size=patch_size, in_chans=3, embed_dim=embedding_dim
)

# 创建一个随机图像
img = torch.rand(batch_size, 3, image_size, image_size)
print(img.shape)

# 将图像转换为序列
patches = patch_embed(img)
print(patches.shape)
# 实例化多头注意力模块
multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

# 应用多头注意力
attn_output, attn_output_weights = multihead_attn(patches, patches, patches)

# 输出的形状将是 (batch_size, num_patches, embed_dim)
print(attn_output.shape)


torch.Size([1, 3, 224, 224])
torch.Size([1, 256, 512])
torch.Size([1, 256, 512])


In [13]:
def hard_shrink_relu(input, lambd=0., epsilon=1e-12):
    output = (F.relu(input - lambd) * input) / (torch.abs(input - lambd) + epsilon)
    return output

In [26]:
a = torch.tensor([[-1, 0.1, 3], [4, 5, 6]])

print(a)
output = hard_shrink_relu(a,lambd=2.)
print(output)


tensor([[-1.0000,  0.1000,  3.0000],
        [ 4.0000,  5.0000,  6.0000]])
tensor([[-0., 0., 3.],
        [4., 5., 6.]])


In [10]:
# torch中的topk的用法，能够返回tensor中维度的最值和索引
a = torch.tensor([[-1, 3, 0.1], [4, 5, 6]])
value, index = a.topk(1, dim=1, largest=True)
index

tensor([[1],
        [2]])

In [12]:
out = value[:,[-1]]# 高级索引，取最后一列
out

tensor([[3.],
        [6.]])

In [13]:
import numpy as np

thres = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])

print("原始数组:")
print(thres)
# 高级索引最后一行
thres_last_column = thres[:, [-1]]

print("\n所有行的最后一列:")
print(thres_last_column)

原始数组:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

所有行的最后一列:
[[3]
 [6]
 [9]]


In [2]:
a=(128//2)//2**4
a

4

In [10]:

a=256 * (4 * (2 ** (0 + 1)) // 2) ** 2
a

4096

In [4]:
def make_window(x, window_size, stride=1, padding=0):
    """
    Args:
        x: (B, C, H, W)
        window_size (int): window size

    Returns:
        windows: (B*N, C, ws, ws)
    """
    B, C, W, H = x.shape
    windows = F.unfold(x, window_size, padding=padding, stride=stride) # B, C * ws ** 2, N, #of windows
    windows = windows.view(B, C, window_size**2, -1) #   B, C, ws**2, N
    windows = windows.permute(0, 3, 1, 2).contiguous().view(-1, C, window_size, window_size) # B*N, C, ws, ws
    
    return windows
def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, C, window_size, window_size)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, C, H, W)
    """
    num_windows = H * W / window_size / window_size
    B = int(windows.shape[0] / num_windows)
    x = windows.view(B, H // window_size, W // window_size, -1, window_size, window_size)
    x = x.permute(0, 3, 1, 4, 2, 5).contiguous().view(B, -1, H, W)
    return x

In [5]:
x= torch.randn(1,3,128,128)
x.size(0)

1

In [6]:
a = 128//2
# x, window_size=W//self.num_patch, stride=W//self.num_patch, padding=0
out = make_window(x, window_size=a, stride=a, padding=0)
out.shape

torch.Size([4, 3, 64, 64])

In [14]:
# x, w, h * self.num_patch, w * self.num_patch
out2 =window_reverse(out, 64, 64*2, 64*2)
x=out2
# x = window_reverse(x,128,128*2,128*2)

## 判别器

In [15]:
class SimpleDiscriminator(nn.Module):
    def __init__(self, num_in_ch=1, size=4, num_layers = 4, inplace=True):
        super(SimpleDiscriminator, self).__init__()

        self.size = size

        keep_stats = True

        out_channels = 16
        layers = [
            nn.Conv2d(num_in_ch, out_channels, 5, 2, 2, bias=True),
            #nn.BatchNorm2d(16, track_running_stats=keep_stats), # this maybe required
            nn.LeakyReLU(0.2, inplace=inplace),
        ]
        
        for ilayer in range(num_layers):
            in_channels = out_channels
            out_channels = min(16 * (2 ** (ilayer + 1)), 256)
            layers.extend([
                nn.Conv2d(in_channels, out_channels, 5, 2, 2, bias=True),
                nn.BatchNorm2d(out_channels, track_running_stats=keep_stats),
                nn.LeakyReLU(0.2, inplace=inplace),
            ])

        self.conv_model = nn.Sequential(*layers)

        self.regressor = nn.Linear(out_channels * size * size, 1)

    def forward(self, img):
        B = img.size(0)

        x = self.conv_model(img) # B, 128, W/16, H/16

        x = x.view(B, -1)
        x = self.regressor(x)
        return x


In [21]:
x= torch.randn(1,3,128,128)
# CONFIG.num_in_ch, size=CONFIG.size, num_layers=CONFIG.num_layers
model_dic = SimpleDiscriminator(num_in_ch=3, size=4, num_layers=4)
out_dic = model_dic(x)
x.shape

torch.Size([1, 3, 128, 128])

In [22]:
out_dic

tensor([[-0.1174]], grad_fn=<AddmmBackward0>)

In [ ]:
def Maduo(nums, c_nums, zuo)